In [1]:
import numpy as np
import os 
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
import tensorflow as tf 
import tensorflow_datasets as tfds
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

In [3]:
batch_size = 32 
img_height = 224
img_width = 224 

In [4]:
def preprocess(image, label):
    final_image = keras.applications.xception.preprocess_input(image)
    return final_image, label

In [5]:
validation_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=None, seed=42)

Found 50947 files belonging to 7 classes.


2022-08-16 09:47:44.363738: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-16 09:47:44.386613: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-16 09:47:44.389533: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-16 09:47:44.393637: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [6]:
training_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=None, seed=42)

Found 399839 files belonging to 7 classes.


In [7]:
validation_ds = validation_s.map(preprocess).batch(batch_size).prefetch(1)
training_ds = training_s.map(preprocess).batch(batch_size).prefetch(1)

In [ ]:
base_model = keras.applications.Xception(input_shape=(224,224,3), include_top = False, weights='imagenet')

In [ ]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer="Adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(training_ds, validation_data=validation_ds, epochs=3)

In [ ]:
for layer in base_model.layers:
    layer.trainable = True

In [ ]:
opt = keras.optimizers.Adam(learning_rate=1e-4)

In [ ]:
es = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

In [ ]:
tf.test.is_gpu_available()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_ds, validation_data=validation_ds, epochs=20, callbacks=[es])